In [1]:
from pyspark import SparkContext #Library
sc = SparkContext("local[*]", "temp") #Start a spart context


In [2]:
posts_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/part/allPosts/*.gz") #create an RDD from file
print posts_lines
posts_lines.count()
#posts_lines.take(8)

MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2


212990

In [3]:
users_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/part/allUsers/*.gz")
print users_lines
users_lines.count()

MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:-2


100425

In [4]:
#Import parsing library.
import xml.etree.ElementTree as ET
import numpy as np


In [5]:
def parseUser(line):
    if '  <row'in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "Id" and "Reputation" in root.attrib:
                return(root.attrib["Id"], int(root.attrib["Reputation"]))
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            

In [6]:
users = users_lines.map(parseUser).filter(lambda x: x!= 'Empty')
print users.count()
print users.take(99)

50320
[('70185', 1), ('70186', 6), ('70187', 1), ('70188', 1), ('70189', 101), ('70190', 1), ('70191', 1), ('70192', 1), ('70193', 1), ('70194', 11), ('70195', 1), ('70196', 6), ('70197', 18), ('70198', 1), ('70199', 51), ('70200', 11), ('70201', 1), ('70202', 101), ('70203', 116), ('70204', 103), ('70205', 6), ('70206', 101), ('70207', 1), ('70208', 101), ('70211', 1), ('70212', 1), ('70213', 1), ('70214', 1), ('70215', 11), ('70216', 1), ('70217', 1), ('70218', 1), ('70219', 101), ('70220', 1), ('70221', 1), ('70222', 1), ('70223', 1), ('70224', 6), ('70225', 1), ('70226', 11), ('70227', 101), ('70228', 1), ('70229', 1), ('70230', 1), ('70231', 1), ('70232', 1), ('70233', 1), ('70234', 1), ('70235', 1), ('70236', 1), ('70237', 3), ('70238', 1), ('70239', 1), ('70240', 1), ('70241', 1), ('70242', 1), ('70243', 6), ('70244', 101), ('70245', 1), ('70246', 1), ('70247', 1), ('70248', 1), ('70249', 1), ('70250', 4), ('70251', 1), ('70252', 1), ('70253', 101), ('70254', 1), ('70255', 1), (

In [7]:
def parsePost(line):
    if '  <row'in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "OwnerUserId" and "PostTypeId" in root.attrib:
                type = root.attrib["PostTypeId"]
                try:
                    owner = root.attrib["OwnerUserId"]
                except:
                    pass
                    return ("Empty")
                if type == '2':
                    type_count = [1.,0.,1.]
                else:
                    if type == '1':
                        type_count = [0.,1.,1.]
                    else:
                        return("Empty")
                return(owner, type_count)
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [8]:
posts = posts_lines.map(parsePost).filter(lambda x: x!= 'Empty')\
        .reduceByKey(lambda x, y: np.add(x,y))
print posts
print posts.take(40)
posts.count()


PythonRDD[12] at RDD at PythonRDD.scala:43
[('23994', [0.0, 1.0, 1.0]), ('61518', array([ 0.,  2.,  2.])), ('35549', [0.0, 1.0, 1.0]), ('69951', [0.0, 1.0, 1.0]), ('29446', [1.0, 0.0, 1.0]), ('11549', array([ 0.,  3.,  3.])), ('21631', [1.0, 0.0, 1.0]), ('60913', [0.0, 1.0, 1.0]), ('17297', [0.0, 1.0, 1.0]), ('11384', array([ 0.,  8.,  8.])), ('55294', [1.0, 0.0, 1.0]), ('43270', [0.0, 1.0, 1.0]), ('10897', [0.0, 1.0, 1.0]), ('5987', [1.0, 0.0, 1.0]), ('22267', array([ 2.,  0.,  2.])), ('48893', array([ 0.,  2.,  2.])), ('3993', [0.0, 1.0, 1.0]), ('61176', [0.0, 1.0, 1.0]), ('56583', array([ 1.,  1.,  2.])), ('53900', [0.0, 1.0, 1.0]), ('25065', [0.0, 1.0, 1.0]), ('56236', array([ 0.,  2.,  2.])), ('12703', [0.0, 1.0, 1.0]), ('61195', [0.0, 1.0, 1.0]), ('43185', array([ 0.,  2.,  2.])), ('35430', [0.0, 1.0, 1.0]), ('42457', array([ 0.,  3.,  3.])), ('31915', [0.0, 1.0, 1.0]), ('37734', array([ 1.,  1.,  2.])), ('5857', [0.0, 1.0, 1.0]), ('55587', [0.0, 1.0, 1.0]), ('69554', [0.0, 1.0, 

26887

In [9]:
joint = posts.join(users).map(lambda x: (x[1][1], x[1][0], int(x[0])))\
        .map(lambda x: (x[0],(x[2], x[1][0]/x[1][2]))).sortByKey(False)\
        .map(lambda x: x[1]).collect()
print joint[:99]


[(919, 0.99669421487603305), (805, 0.99597495527728086), (686, 0.98030495552731889), (7290, 0.99188876013904981), (930, 0.9817351598173516), (4505, 1.0), (4253, 0.99097472924187724), (183, 0.84787018255578095), (11032, 0.98756476683937822), (28746, 0.96842105263157896), (887, 0.97948717948717945), (159, 0.97288135593220337), (2116, 0.98333333333333328), (4856, 0.95431472081218272), (22047, 1.0), (5739, 0.98727735368956748), (3277, 0.95608108108108103), (88, 0.96604938271604934), (2970, 1.0), (601, 0.97721518987341771), (17230, 0.99700598802395213), (449, 1.0), (2392, 0.97241379310344822), (1390, 0.94117647058823528), (5836, 0.84644194756554303), (7555, 1.0), (603, 0.81588447653429608), (7972, 0.98230088495575218), (6633, 0.99122807017543857), (2958, 0.99303135888501737), (9394, 0.97008547008547008), (7828, 0.9850427350427351), (2817, 0.82068965517241377), (7224, 0.97575757575757571), (4598, 0.98571428571428577), (7071, 0.9107142857142857), (1739, 0.99487179487179489), (1036, 0.95454545

In [10]:
total = posts.map(lambda x: ("t",x[1])).reduceByKey(lambda x, y: np.add(x,y)).collect()
total_sum = total[0]
fraction = (-1, total_sum[1][0]/total_sum[1][2])
print fraction

(-1, 0.51576399852522714)


In [ ]:
[(7071, 0.9107142857142857)] * 100